Generate new sentences and remap the newly created sentences to orignial corresponding ID from train and test dataset.

#### Compress

Create new sentences of length (~450) by joining them to make it longer.

In [1]:
import pandas as pd
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import re
from matplotlib.pyplot import figure
import numpy as np
from wordcloud import WordCloud, STOPWORDS
import unicodedata

#note
#check noIds

def splitsentence(df,long=True):
    if long:
        if df['wordcount']>450:
            # at least 2 sids after concatenation
            if len(str(df['sid']))>15:
                num=int(len(df['sentence'])/2)
                sentence=["".join(df['sentence'][:num]),"".join(df['sentence'][num:])]
            else: #
                num=int(df['sentence'][0].count(" ")/2)
                txt=df['sentence'][0].split(' ')
                sentence=[' '.join(txt[: num]), ' '.join(txt[num: ])]
        else:
            sentence= ["".join(df['sentence'])]
    else:
            if len(str(df['sid']))>15:
                L=2
                S =1 
                a=np.array(df['sentence'])
                nrows = ((a.size-L)//S)+1
                sentence=a[S*np.arange(nrows)[:,None] + np.arange(L)]
                sentence=["".join(s) for s in sentence]
            else: #
                sentence =df['sentence']

    return sentence


In [4]:
# df=pd.read_csv("../3TrainCompressManualClean.tsv", sep='\t')
# df['sentence'] = df['sentence'].apply(lambda x: unicodedata.normalize("NFKD",x)) 
# df['sentence'] = df['sentence'].apply(lambda x: ' '.join(x.split())) # remove multiple space
# df['title'] = df['title'].apply(lambda x: unicodedata.normalize("NFKD",x)) 
# df['title'] = df['title'].apply(lambda x: ' '.join(x.split())) # remove multiple space
# df=df.sort_values(['html_id', 'sid'], ascending=[True, True]).reset_index(drop=True)
# df.to_csv("3TrainCompressManualClean_space.tsv", sep='\t')

In [ ]:
# df=pd.read_pickle("../zfinal_testdf.pkl")

# noidshtml=df[df['paragraph_id']=='NoID']['html_id'].value_counts().index.tolist()
# pl=list(set(df.html_id.value_counts().index.tolist())-set(noidshtml))
# print(pl[:10])
# print(pl[10:20])
# print(pl[20:30])
# plSet=pd.DataFrame()
# plSet[0]=pl[:10]
# plSet[1]=pl[10:20]
# plSet[2]=pl[20:30]
# plSet[3]=list(noidshtml)+pl[:2]
# plSet.to_pickle("PLSets.pkl")

#### Concat Paragraph

Concat similar sentences according to same paragraph id, same label, same id group etc.</br>
Remap the newly created sentences data with id from training and testing file.</br>
Create for train data(+label),test data(-label),validation data(-label).

In [ ]:
#testparagraphconcat
def extendParaConcat(df,uselabel=False):
    print("before  :" ,df.shape)
    if uselabel:
        df = df.set_index(['uid','sentence','html_id','title','label'])['sid'].apply(pd.Series).stack().reset_index()
    else:
        df = df.set_index(['uid','sentence','html_id','title'])['sid'].apply(pd.Series).stack().reset_index()
        df=df.rename(columns={0:"sid"})
        display(df.head(3))
        df=df.sort_values(['html_id', 'sid'], ascending=[True, True]).reset_index(drop=True)

        
    print("after  :" ,df.shape)
    return df
    
def paraConcat(df,uselabel=False):
    print("before",df.shape)
    
    dfID=df[df['paragraph_id']!='NoID']
    # df.to_csv("1testTP_space.tsv", sep='\t')
    dfID['sentence'] = dfID['sentence'].apply(lambda x: unicodedata.normalize("NFKD",x)) 
    dfID['sentence'] =dfID['sentence'].apply(lambda x: str(x).strip())
    dfID['uid']=dfID['html_id']+dfID['title'].apply(lambda x:"".join([w[0] for w in x.split() if w[0].isalnum() ]))+dfID['paragraph_id']
    dfID['wordcount']=dfID['sentence'].apply(lambda x:len(str(x).split()))
    if uselabel:
        dfID=(dfID.groupby(['uid'])
          .agg({'title':'first','html_id':'first','label':pd.Series.mode,'sentence':''.join,'sid':lambda x: list(x.values)})
            .reset_index('uid'))
        dfID.label=dfID.label.apply(lambda x:x[-1] if type(x) is np.ndarray else x)
        print(dfID.label.value_counts())
    else:
        dfID=(dfID.groupby(['uid'])
          .agg({'title':'first','html_id':'first','sentence':''.join,'sid':lambda x: list(x.values)})
            .reset_index('uid'))
    dfNID=df[df['paragraph_id']=='NoID']
    dfNID.sid=dfNID.sid.apply(lambda x:list(str(x).split()))
    df=pd.concat([dfID,dfNID])
    print("after",df.shape)
    print("need to do extendParaConcat")
    return df
    
# traindf=pd.read_pickle("../ztraindf.pkl")
# traindf=paraConcat(traindf,uselabel=True) # same for val and test
# traindf=extendParaConcat(traindf)
# traindf.to_csv("5Train_ParaConcat2"+".tsv", sep='\t')
# traindf.to_pickle("5Train_ParaConcat2"+".pkl")

    
Ftestdf=pd.read_pickle("../zfinal_testdf.pkl")
Ftestdf=paraConcat(Ftestdf,uselabel=False) # same for val and test
# Ftestdf=extendParaConcat(Ftestdf)

# Ftestdf.to_csv("5FtestParaConcat"+".tsv", sep='\t')
Ftestdf.to_pickle("5Ftest_ParaConcat"+".pkl")

# testdf=pd.read_pickle("../ztestdf.pkl")
# valdf=pd.read_pickle("../ztraindf.pkl")

# paraConcat(testdf,"5test_ParaConcat")
# paraConcat(valdf,"5val_ParaConcat")


# paraConcat(valdf,"")



In [ ]:
df=pd.read_pickle("../traindf.pkl")
df['sentence'] = df['sentence'].apply(lambda x: unicodedata.normalize("NFKD",x)) 

df['sentence'] =df['sentence'].apply(lambda x: str(x).strip())

print("make uid")
df['uid']=df['html_id']+df['title'].apply(lambda x:"".join([w[0] for w in x.split() if w[0].isalnum() ]))+df['paragraph_id']
df['wordcount']=df['sentence'].apply(lambda x:len(str(x).split()))
df['sentcount_beforesplit']=df['sentence'].apply(lambda x:len(x))
print("df shape ",df.shape)
print("exclude title")
notitle=df.loc[(df['isTitle']==0) | (df['wordcount']> 400)].reset_index()

print("notitle shape ",notitle.shape)
display(notitle.head(3))
print("concat consecutive value group by uid")
blocks = notitle['label'].ne(notitle['label'].shift()).cumsum()
notitle=(notitle.groupby(['uid', blocks])
  .agg({'label':'first','title':'first','html_id':'first','sentence':lambda x: list(x.values),'sid':','.join})
  .reset_index('label', drop=True)
)
notitle['wordcount']=notitle['sentence'].apply(lambda x:len(str(x).split()))

notitle.reset_index(inplace=True)
print("no title")
display(notitle.head(3))

print("Shink 0 : otherdf")
otherdf=notitle[notitle.label==0]
print(otherdf.shape)
print("Split sentence")
otherdf['sentcount_beforesplit']=otherdf['sentence'].apply(lambda x:len(x))
print("otherdf.shape before splitsentence ",otherdf['sentence'].apply(lambda x:len(x)).value_counts())
otherdf['sentence']=otherdf.apply(splitsentence,axis=1)
print("otherdf.shape after splitsentence ",otherdf['sentence'].apply(lambda x:len(x)).value_counts())
display(otherdf.head(3))

print("Shink 123 : esgdf1")

esgdf1=notitle[notitle.label!=0]
print(esgdf1.shape)
esgdf1['sentcount_beforesplit']=esgdf1['sentence'].apply(lambda x:len(x))
print("esgdf1.shape before splitsentence ",esgdf1['sentence'].apply(lambda x:len(x)).value_counts())
esgdf1['sentence']=esgdf1.apply(splitsentence,axis=1)
print("esgdf1.shape after splitsentence ",esgdf1['sentence'].apply(lambda x:len(x)).value_counts())
display(esgdf1.head(3))

print("2 pairs 123 : esgdf2")

esgdf2=notitle[notitle.label!=0]
print(esgdf2.shape)
esgdf2['sentcount_beforesplit']=esgdf2['sentence'].apply(lambda x:len(x))
print("esgdf2.shape before splitsentence ",esgdf2['sentence'].apply(lambda x:len(x)).value_counts())
esgdf2['sentence']=esgdf2.apply(splitsentence,long=False,axis=1)
print("esgdf2.shape after splitsentence ",esgdf2['sentence'].apply(lambda x:len(x)).value_counts())
display(esgdf2.head(3))


In [ ]:
def extenddf(df):
    print("before  :" ,df.shape)
    df = df.set_index(['label','uid','sid','html_id','title'])['sentence'].apply(pd.Series).stack()
    df = df.reset_index()
    df=df.rename(columns={0:"sentence"})
    display(df.head(3))
    return df
    
    
otherdf=extenddf(otherdf)
print("after  :" ,otherdf.shape)
esgdf1=extenddf(esgdf1)
print("after  :" ,esgdf1.shape)
esgdf2=extenddf(esgdf2)
print("after  :" ,esgdf2.shape)

print("original shape : ",len(df.index))
print("original shape 0 :",len(df[df.label==0].index))
print("title shape 0 :",len(df[(df.label==0) & (df.isTitle==1)]))
print("title shape 123 :",len(df[(df.label!=0) & (df.isTitle==1)]))

print("")
print("otherdf shape 0:",len(otherdf.index))

print("original shape: 123",df[df.label!=0].shape)
print("esg1 shape 123:",len(esgdf1.index))
print("esg2 shape 123:",len(esgdf2.index))
print("esg total:",len(esgdf1.index)+len(esgdf2.index)+len(df[(df.label!=0) & (df.isTitle==1)]))

print("Concat data")
columns=["uid","label","sentence","title","html_id","sid"] #
traindf=pd.concat([otherdf[columns],esgdf1[columns],esgdf2[columns]])
set(traindf['uid']) == set(traindf['uid'].values)
traindf=pd.concat([traindf,df[df.isTitle==1][columns]])
print(traindf.shape)

traindf=traindf.sort_values(['html_id', 'sid'], ascending=[True, True]).reset_index(drop=True)
traindf.to_csv("compressTrain_title.tsv", sep='\t')

#### CONCATAFTER
 
Concatenate with only one next sentences.</br>
check if two are ok to joined - same label, have same table, ending periods and start Capitial letter, etc.

In [12]:
def doConcat(df,uselabel=False):
    global duplicatelist
    sen2flg=1
    sen3flg=1
    senlist=[df.sentence]
    sidlist=[df.sid]
    
    if uselabel:
        if df.label != df.label2:
            return df.sentence,sidlist
        if df.label != df.label3:
            sen3flg = 0
            
    if df.title != df.title2:
        return df.sentence,sidlist
    
    if not ((df.start_cap==True ) and (df.hasfullstop==False or df.last_colon==True)):
        return df.sentence,sidlist
    else:
        # if too many cap , assume title and return
        words=" ".join([w for w in df.sentence.split() if not w.lower() in STOPWORDS])
        capcount=sum(1 for w in str(words).split() if w[0].isupper())
        if capcount>=len(words.split()):
            return df.sentence,sidlist

    if df.random2:
        sen2flg=0
    if (df.random3 or df.sentence2[-1]=='.' or df.sentence3[0].isupper()) :
        sen3flg=0

    if sen2flg==1:
        senlist.append(df.sentence2)
        sidlist.append(df.sid2)
        duplicatelist.append(df.sid2)
    if sen3flg==1:
        senlist.append(df.sentence3)
        sidlist.append(df.sid3)
        duplicatelist.append(df.sid3)
    sentence = ' '.join(senlist)
    
    return sentence,sidlist

#### Concat previous sentence + current sentence + after sentence.

check if two are ok to joined - same label, have same table, ending periods and start Capitial letter, etc.

In [23]:
traindf=pd.read_pickle("../ztraindf.pkl")
traindf.shape

(65963, 7)

In [ ]:

def concatCSS(df,outpufilename,uselabel=False):
    global duplicatelist
    df['sentence'] = df['sentence'].apply(lambda x: unicodedata.normalize("NFKD",x)) 
    df['sentence'] = df['sentence'].apply(lambda x: ' '.join(x.split())) # remove multiple space
#     df['suid']=df['html_id']+df['sid']
    
    tdf=df.copy()
    tdf['hasfullstop'] = tdf['sentence'].apply(lambda x: str(x)[-1]==".")
    tdf['wordcount']=tdf['sentence'].apply(lambda x:len(str(x).split()))
    tdf['stop_word_count'] = tdf['sentence'].apply(lambda x: len([w for w in str(x).lower().split() if w in STOPWORDS]))
    tdf['cap_word_per'] = tdf['sentence'].apply(lambda x: sum(1 for w in str(x).split() if w[0].isupper())/len(str(x).split()))
    tdf['start_num'] = tdf['sentence'].apply(lambda x: str(x)[0].isdigit())
    tdf['start_quote']=tdf['sentence'].apply(lambda x: str(x)[0]=='\"')
    tdf['last_colon'] = tdf['sentence'].apply(lambda x: str(x)[-1]==":")    
    tdf['start_cap'] = tdf['sentence'].apply(lambda x: str(x)[0].isupper())
    tdf['start_isalnum'] = tdf['sentence'].apply(lambda x:str(x)[0].isalnum())
    tdf['random'] =tdf['sentence'].apply(lambda x:((x=="") or (len(x)-len(re.findall("[a-zA-Z$%]", x)))>=len(re.findall("[a-zA-Z$%]", x))))
    
    if uselabel:
        cols=['sentence','isTitle','title','label','sid','start_cap','random']
        tdf[['sentence2','isTitle2','title2','label2','sid2','start_cap2','random2']]=tdf[cols].shift(periods=-1, fill_value=np.nan)
        tdf[['sentence3','isTitle3','title3','label3','sid3','start_cap3','random3']]=tdf[cols].shift(periods=-2, fill_value=np.nan)
    else:
        cols=['sentence','isTitle','title','sid','start_cap','random']
        tdf[['sentence2','isTitle2','title2','sid2','start_cap2','random2']]=tdf[cols].shift(periods=-1, fill_value=np.nan)
        tdf[['sentence3','isTitle3','title3','sid3','start_cap3','random3']]=tdf[cols].shift(periods=-2, fill_value=np.nan)
    
    tdf.reset_index(inplace=True,drop=True)
    oklist=[]
    oklist.extend(tdf[(tdf.start_cap==1)& (tdf.hasfullstop==1)].sentence.index.tolist())
    oklist.extend(tdf[tdf.isTitle==1].sentence.index.tolist())
    oklist.extend(tdf[tdf.start_num==1].sentence.index.tolist())
    oklist.extend(tdf[tdf.random==True].sentence.index.tolist())
    oklist=list(set(oklist))
    tdf=tdf.drop(tdf.index[oklist])
    tdf.reset_index(inplace=True,drop=True)
    print("afterdroping oklist",tdf.shape)
    
    ######
    print("duplicatelist start")
    print(duplicatelist)

    print("tdf.shape :",tdf.shape)
    print(uselabel)
    tdf[['newsentence','sidlist']]=tdf.apply(doConcat,uselabel=uselabel,axis=1, result_type ='expand')  ####### diff
    
    tdf.to_csv("before stacking.tsv", sep='\t', index=False) 
    pass
    if uselabel:
        tdf.drop(columns=['sentence', 'sentence2','isTitle2','title2','label2','sid2','start_cap2','random2','sentence3','isTitle3','title3','label3','sid3','start_cap3','random3'],inplace=True)
    else:
        tdf.drop(columns=['sentence', 'sentence2','isTitle2','title2','sid2','start_cap2','random2','sentence3','isTitle3','title3','sid3','start_cap3','random3'],inplace=True)

    tdf=tdf.rename({'newsentence': 'sentence'}, axis=1) 
    print("tdf.shape :",tdf.shape)
    
    duplicatelist=list(set(duplicatelist))
    print("duplicatelist delete")
    print(len(duplicatelist))
    display(tdf.head(4))
    tdf=tdf.drop(tdf[tdf.sid.isin(duplicatelist)].index.tolist())
    tdf.reset_index(inplace=True,drop=True)
    remain_idx=tdf.index.tolist()

    print("after tdf.shape :",tdf.shape)
    print("before stacking.csv")
    display(tdf.head(4))
    if uselabel:
        tdf = tdf.set_index(['sentence', 'html_id','label','title','isTitle','paragraph_id'])['sidlist'].apply(pd.Series).stack()
    else:
        tdf = tdf.set_index(['sentence', 'html_id','title','isTitle','paragraph_id'])['sidlist'].apply(pd.Series).stack()
    tdf=tdf.reset_index()
    tdf=tdf.rename(columns={0:"sid"})
    print("after stacking")
    display(tdf)
    
    print("after tdf.shape :",tdf.shape)
    
    df=df[~df.sid.isin(tdf.sid.tolist())]
    if uselabel:
        col=['sid','html_id','sentence','label','title','isTitle','paragraph_id']
        df=pd.concat([tdf[col],df[col]], axis=0)
    else:
        col=['sid','html_id','sentence','title','isTitle','paragraph_id']
        df=pd.concat([tdf[col],df[col]], axis=0)
    
    print('after concat with ok list:',df.shape)
    df=df.sort_values(['sid'], ascending=True).reset_index(drop=True)
    df.to_csv(outpufilename, sep='\t', index=False) 
#     df.to_csv(outpufilename, sep='\t', index=False)  ####### diff
duplicatelist=[]
FNtestdf=pd.read_pickle("../zfinal_testdf.pkl")
concatCSS(FNtestdf,"ConcatFNTest.tsv",uselabel=False)

In [ ]:
duplicatelist=[]
traindf=pd.read_pickle("../traindf.pkl")
concatCSS(traindf,"ConcatTrain.tsv",uselabel=True)

duplicatelist=[]
valdf=pd.read_pickle("../traindf.pkl")
valdf=valdf.drop(['label'],axis=1)
concatCSS(valdf,"ConcatVal.tsv",uselabel=False)

duplicatelist=[]
testdf=pd.read_pickle("../testdf.pkl")
concatCSS(testdf,"ConcatTest.tsv",uselabel=False)


duplicatelist=[]
FNtestdf=pd.read_pickle("../zfinal_testdf.pkl")
concatCSS(FNtestdf,"ConcatFNTest.tsv",uselabel=False)

#### Current+After or Current+ Before sentence

 for training model with 2 sentences.

In [25]:
#Train
    

def combineAB(df):
    if df['bsentence']==df['sentence']:
        df['bsentence']=""
    if df['asentence']==df['sentence']:
        df['asentence']=""
    return df['bsentence']+" "+df['sentence']+" "+df['asentence']

def makeBAsentence(df,uselabel=False):
    asentence=""
    bsentence=""
    if df['pid']==df['pid']:
        if uselabel:
            if df['label'] == df['blabel']:
                bsentence=df['bsentence']
        else:
            bsentence=df['bsentence']
    if df['tid']== df['atid']:
        if uselabel:
            if df['label'] == df['alabel']:
                asentence=df['asentence']
        else:
            asentence=df['asentence']
    return asentence,bsentence
    
def BCA(df,filename,uselabel=False):
    df['sentence'] = df['sentence'].apply(lambda x: unicodedata.normalize("NFKD",x)) 
    df['sentence'] = df['sentence'].apply(lambda x: ' '.join(x.split())) # remove multiple space
    print("make uid")
    
    df['pid']=df['html_id']+df['paragraph_id'] 
    df['tid']=df['html_id']+df['title']
    
    tdf=df.copy()
    tdf['random'] =tdf['sentence'].apply(lambda x:((x=="") or (len(x)-len(re.findall("[a-zA-Z$%]", x)))>=len(re.findall("[a-zA-Z$%]", x))))
    
    
    tdf.reset_index(inplace=True,drop=True)
    oklist=[]
    oklist.extend(tdf[tdf.isTitle==1].sentence.index.tolist())
    oklist.extend(tdf[tdf.random==True].sentence.index.tolist())
    oklist=list(set(oklist))
    tdf=tdf.drop(tdf.index[oklist])
    
    if uselabel:
        cols=['sentence','isTitle','title','label','pid','tid','random']
        tdf[['asentence','aisTitle','atitle','alabel','apid','atid','arandom']]=tdf[cols].shift(periods=-1, fill_value=np.nan)
        tdf[['bsentence','bisTitle','btitle','blabel','bpid','btid','brandom']]=tdf[cols].shift(periods=1, fill_value=np.nan)
    else:
        cols=['sentence','isTitle','title','pid','tid','random']
        tdf[['asentence','aisTitle','atitle','apid','atid','arandom']]=tdf[cols].shift(periods=-1, fill_value=np.nan)
        tdf[['bsentence','bisTitle','btitle','apid','atid','brandom']]=tdf[cols].shift(periods=1, fill_value=np.nan)

        
    lbcolumns= [col for col in tdf if 'label' in col]
    
    tdf[[col for col in tdf if 'sentence' in col]]=tdf[[col for col in tdf if 'sentence' in col]].fillna("")
    tdf[[col for col in tdf if 'title' in col]]=tdf[[col for col in tdf if 'title' in col]].fillna("")
    tdf[[col for col in tdf if 'pid' in col]]=tdf[[col for col in tdf if 'pid' in col]].fillna("")
    tdf[[col for col in tdf if 'tid' in col]]=tdf[[col for col in tdf if 'tid' in col]].fillna("")
    tdf[[col for col in tdf if 'random' in col]]=tdf[[col for col in tdf if 'random' in col]].fillna("")
    tdf[[col for col in tdf if 'isTitle' in col]]=tdf[[col for col in tdf if 'isTitle' in col]].fillna(0)

    if tdf[lbcolumns].isnull().values.any():
        tdf[lbcolumns]=tdf[lbcolumns].fillna(0)
        for col in lbcolumns:
            tdf[col] = tdf[col].astype('int')
            
    tdf.reset_index(inplace=True,drop=True)
    tdf[['aasentence','bbsentence']]=tdf.apply(makeBAsentence,uselabel=uselabel,axis=1, result_type ='expand')  ####### diff
    tdf.drop(columns=['asentence', 'bsentence'],inplace=True)
    tdf=tdf.rename(columns={"aasentence":"asentence","bbsentence":"bsentence"})
    tdf['Lsentence']=tdf.apply(combineAB,axis=1) 
    tdf['Lsentence'].apply(lambda x: ' '.join(x.split())) 
    tdf=tdf.rename(columns={"sentence":"sentence_ori"}).rename(columns={"Lsentence":"sentence"})

    concatdf=pd.concat([df[df.index.isin(df.index[oklist])],tdf])
    concatdf=concatdf.sort_index()
    concatdf=concatdf.sort_values(['sid'], ascending=True).reset_index(drop=True)
    print(concatdf.shape)
    concatdf.to_csv(filename, sep='\t', index=False) 


#### Concat previous sentence + current sentence + after sentence.

check if two are ok to joined - same label, have same table, ending periods and start Capitial letter, etc.

In [81]:
df=pd.read_csv("ConcatTrain.tsv", sep='\t') #62sentenceTest|_4TestCat
BCA(df,"BCACatTrain.tsv",uselabel=True)
df=pd.read_csv("ConcatVal.tsv", sep='\t') 
BCA(df,"BCACatVal.tsv")
df=pd.read_csv("ConcatTest.tsv",sep='\t') 
BCA(df,"BCACatTest.tsv")

make uid
(65963, 26)
make uid
(65963, 21)
make uid
(28437, 21)


In [26]:
df=pd.read_csv("ConcatFNTest.tsv",sep='\t') 
BCA(df,"BCACatFNTest.tsv")

make uid
(60507, 20)
